# REPASO CURSO DE DICIEMBRE 2019 BIG-DATA

In [ ]:
# Preparando el entorno para Jupiter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

In [2]:
# importamos pandas para trabajar con dataframes
import pandas as pd
from pandas import DataFrame

In [ ]:
#importamos la libreria de Scrapy que sera la que usaremos para hacer web scraping

try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
    
from scrapy.crawler import CrawlerProcess

In [ ]:
# como paso intermedio guardaremos la informacion en un archivo json para luego convertilo en un dataframe


import json

# Esta clase creara un pipeline que escribira la info encontrada en un archivo json


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
    
    
# pipeline: serie de cariables y otras propiedades

In [ ]:
#definiendo la "araña" 
#la clase QuotesSpider definirá de cual url se trae la informacion y que valores regresará.
#para evitar infinidades de mensajes de como se esta realizando el procedimiento se agrega el Log lvl en Warning


import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

In [ ]:
#iniciamos nuestro Crawler


process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

In [ ]:
# creamos el DataFrame

dfjl = pd.read_json('quoteresult.jl', lines=True)

In [ ]:
df = DataFrame(dfjl)

In [ ]:
#exportamos nuestra informacion recopiladas indicando el formato y la ruta


df.to_csv(r'Desktop\Frases.csv')
